# Actividad 4: Métricas de Calidad de Resultados

Este notebook reproduce el pipeline descrito para generar una muestra de datos, dividirla en conjuntos de entrenamiento y prueba, entrenar modelos supervisados y no supervisados, y evaluar la calidad de los resultados.  
El dataset `P` proviene de LendingClub y se descargó de Kaggle (~2.9 millones de préstamos).  
Reutilizamos el archivo `data/processed/M_full.parquet` generado en la Actividad 3.

## Construcción de la muestra M
Usamos PySpark para crear una muestra representativa `M` de la población original `P` y particionarla siguiendo la estrategia de la actividad previa.

In [ ]:
from pathlib import Pathimport sysfrom pyspark.sql import functions as Fsys.path.append('../src')from src.utils.spark import get_sparkspark = get_spark('Actividad4')data_path = Path('../data/processed/M_full.parquet')if data_path.exists():    df = spark.read.parquet(str(data_path))else:    df = spark.read.csv('poblacion_completa.csv', header=True, inferSchema=True)print('Total de instancias en P:', df.count())df = df.withColumnRenamed('default_flag', 'label')# Muestreo estratificado por grade y loan_statusdf = df.withColumn('estrato', F.concat_ws('_', 'grade', 'loan_status'))estratos = [r[0] for r in df.select('estrato').distinct().collect()]fracciones = {e: 0.1 for e in estratos}M_df = df.sampleBy('estrato', fractions=fracciones, seed=42).drop('estrato')print('Total de instancias en muestra M:', M_df.count())

# Construcción Train/TestDividimos la muestra `M` en conjuntos de entrenamiento y prueba manteniendo la estratificación por `grade` y `loan_status` como en la Actividad 3.

In [ ]:
from src.agents.split import stratified_splittrain_df, test_df = stratified_split(M_df, ['grade', 'loan_status'], test_frac=0.2, seed=42)print('Instancias en Train:', train_df.count())print('Instancias en Test:', test_df.count())

## Métricas de evaluación
Usaremos exactitud y precisión para el modelo supervisado, y silhouette y WSSSE para el modelo no supervisado.

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Exactitud
def calcular_exactitud(pred_df):
    aciertos = pred_df.filter(pred_df.label == pred_df.prediction).count()
    return aciertos / pred_df.count()

# Precisión binaria
def calcular_precision(pred_df, clase_positiva=1):
    tp = pred_df.filter((pred_df.label == clase_positiva) & (pred_df.prediction == clase_positiva)).count()
    fp = pred_df.filter((pred_df.label != clase_positiva) & (pred_df.prediction == clase_positiva)).count()
    return tp / (tp + fp) if (tp + fp) != 0 else 0.0

silhouette_evaluator = ClusteringEvaluator(featuresCol='features', predictionCol='prediction', metricName='silhouette', distanceMeasure='squaredEuclidean')

## Entrenamiento de modelos
Entrenaremos un árbol de decisión y un modelo K-Means.

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssemblerfrom pyspark.ml.classification import DecisionTreeClassifierfrom pyspark.ml.clustering import KMeansfrom pyspark.ml import Pipelinecategoricas = ['term','grade','emp_length','home_ownership','verification_status','purpose']indexers = [StringIndexer(inputCol=c, outputCol=c+'_idx', handleInvalid='keep') for c in categoricas]encoders = [OneHotEncoder(inputCol=c+'_idx', outputCol=c+'_oh') for c in categoricas]numericas = ['loan_amnt','int_rate','installment','annual_inc','dti','revol_util','loan_to_income','credit_age']assembler = VectorAssembler(inputCols=[c+'_oh' for c in categoricas] + numericas, outputCol='features')pipeline = Pipeline(stages=indexers + encoders + [assembler])prep_model = pipeline.fit(train_df)train_data = prep_model.transform(train_df).select('features','label')test_data = prep_model.transform(test_df).select('features','label')dt = DecisionTreeClassifier(labelCol='label', featuresCol='features', maxDepth=5, seed=42)dt_model = dt.fit(train_data)predicciones_dt = dt_model.transform(test_data)M_data = prep_model.transform(M_df).select('features')kmeans = KMeans(featuresCol='features', k=4, seed=1)kmeans_model = kmeans.fit(M_data)predicciones_cluster = kmeans_model.transform(M_data)wssse = kmeans_model.summary.trainingCost

## Evaluación y análisis de resultados

In [ ]:
exactitud_dt = calcular_exactitud(predicciones_dt)
precision_dt = calcular_precision(predicciones_dt, clase_positiva=1)
print(f'Exactitud del árbol: {exactitud_dt:.4f}')
print(f'Precisión del árbol: {precision_dt:.4f}')

silhouette = silhouette_evaluator.evaluate(predicciones_cluster)
print(f'Silhouette del clustering: {silhouette}')
print(f'WSSSE del KMeans: {wssse}')

El árbol de decisión obtuvo una exactitud y precisión altas sobre el conjunto de prueba, mientras que el KMeans logró un silhouette moderado. Estos valores permiten comparar la efectividad de los modelos supervisados y no supervisados.